In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pykml
#yolo쉨 함 불러와봐?
!git clone https://github.com/ultralytics/yolov5
!cd yolov5
!pip install -r yolov5/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 1.4 MB/s eta 0:00:00
Cloning into 'yolov5'...
remote: Enumerating objects: 15821, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 15821 (delta 14), reused 26 (delta 10), pack-reused 15768
Receiving objects: 100% (15821/15821), 14.65 MiB | 26.55 MiB/s, done.
Resolving deltas: 100% (10826/10826), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.6/606.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00


In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [ ]:
#파일 위치
import os
train_images_path = '/content/drive/MyDrive/container_dataset/train_images'
train_labels_path = '/content/drive/MyDrive/container_dataset/train_labels'
valid_images_path = '/content/drive/MyDrive/container_dataset/valid_images'

In [ ]:
import json
import os
from google.colab import drive

# 클래스 이름과 해당 클래스 번호를 매핑하는 딕셔너리
class_mapping = {
    "container": 0,  # 예시를 위해 "container" 클래스를 0으로 매핑
}

def convert_to_yolo_format(json_data, image_width, image_height):
    yolo_labels = []
    features = json_data["features"]

    for feature in features:
        properties = feature["properties"]
        object_imcoords = properties["object_imcoords"].split(",")
        image_id = properties["image_id"]

        # 이미지 크기를 사용하여 바운딩 박스의 중심 좌표와 크기를 구함
        x_center = (float(object_imcoords[0]) + float(object_imcoords[4])) / 2
        y_center = (float(object_imcoords[1]) + float(object_imcoords[5])) / 2
        width = abs(float(object_imcoords[0]) - float(object_imcoords[4]))
        height = abs(float(object_imcoords[1]) - float(object_imcoords[5]))

        # 클래스 번호
        class_name = properties["type_name"]
        class_number = class_mapping.get(class_name)

        if class_number is not None:
            # YOLO 형식으로 변환하여 리스트에 추가
            yolo_label = f"{class_number} {x_center / image_width:.6f} {y_center / image_height:.6f} {width / image_width:.6f} {height / image_height:.6f}"
            yolo_labels.append(yolo_label)

    return yolo_labels

def convert_jsons_to_yolo_labels(json_directory, output_directory, image_width, image_height):
    for filename in os.listdir(json_directory):
        if filename.endswith(".json"):
            json_path = os.path.join(json_directory, filename)
            with open(json_path, "r") as json_file:
                json_data = json.load(json_file)

            # YOLOv5에서 사용할 수 있는 txt 파일 형식으로 변환
            yolo_labels = convert_to_yolo_format(json_data, image_width, image_height)

            # 이미지 파일명에 맞는 txt 파일명 생성
            image_id = json_data["features"][0]["properties"]["image_id"]
            txt_filename = f"{image_id.split('.')[0]}.txt"
            txt_path = os.path.join(output_directory, txt_filename)

            # 변환된 레이블을 txt 파일로 저장
            with open(txt_path, "w") as txt_file:
                for label in yolo_labels:
                    txt_file.write(label + "\n")

            print(f"JSON 파일 {filename}에 대한 YOLO 레이블이 {txt_path} 파일에 저장되었습니다.")


# JSON 파일이 있는 디렉토리 경로 설정 (구글 코랩에서는 파일 업로드로 디렉토리 지정)
json_directory_path = train_labels_path  # 예시 경로, 변경 가능

# YOLO 레이블을 저장할 디렉토리 경로 설정
output_directory_path = "/content/drive/MyDrive/ConSet/traintxt"  # 예시 경로, 변경 가능

# 이미지의 너비와 높이 설정 (실제 이미지 크기에 맞게 설정)
image_width = 1024
image_height = 1024

# JSON 파일들을 YOLO 레이블로 변환하여 텍스트 파일로 저장
convert_jsons_to_yolo_labels(json_directory_path, output_directory_path, image_width, image_height)


JSON 파일 OBJ05618_PS3_K3A_NIA0358.json에 대한 YOLO 레이블이 /content/drive/MyDrive/ConSet/traintxt/OBJ05618_PS3_K3A_NIA0358.txt 파일에 저장되었습니다.
JSON 파일 OBJ05124_PS3_K3A_NIA0334.json에 대한 YOLO 레이블이 /content/drive/MyDrive/ConSet/traintxt/OBJ05124_PS3_K3A_NIA0334.txt 파일에 저장되었습니다.
JSON 파일 OBJ04026_PS3_K3A_NIA0169.json에 대한 YOLO 레이블이 /content/drive/MyDrive/ConSet/traintxt/OBJ04026_PS3_K3A_NIA0169.txt 파일에 저장되었습니다.
JSON 파일 OBJ04996_PS3_K3A_NIA0328.json에 대한 YOLO 레이블이 /content/drive/MyDrive/ConSet/traintxt/OBJ04996_PS3_K3A_NIA0328.txt 파일에 저장되었습니다.
JSON 파일 OBJ04853_PS3_K3A_NIA0317.json에 대한 YOLO 레이블이 /content/drive/MyDrive/ConSet/traintxt/OBJ04853_PS3_K3A_NIA0317.txt 파일에 저장되었습니다.
JSON 파일 OBJ04498_PS3_K3A_NIA0296.json에 대한 YOLO 레이블이 /content/drive/MyDrive/ConSet/traintxt/OBJ04498_PS3_K3A_NIA0296.txt 파일에 저장되었습니다.
JSON 파일 OBJ03626_PS3_K3A_NIA0151.json에 대한 YOLO 레이블이 /content/drive/MyDrive/ConSet/traintxt/OBJ03626_PS3_K3A_NIA0151.txt 파일에 저장되었습니다.
JSON 파일 OBJ04860_PS3_K3A_NIA0317.json에 대한 YOLO 레이블이 /content/drive/My

In [ ]:
import os
import glob

# 디렉터리 경로
directory_path = "/content/drive/MyDrive/container_dataset/train_images"

# 디렉터리 내의 PNG 파일 목록 얻기
png_files = glob.glob(os.path.join(directory_path, "*.png"))

# png 파일들만 따로 저장할 경로 설정
output_directory = "/content/drive/MyDrive/ConSet/Contrain"

# 저장할 디렉토리 생성 (이미 존재하는 경우 생략 가능)
os.makedirs(output_directory, exist_ok=True)

# png 파일들을 지정한 디렉토리에 저장
for png_file in png_files:
    # 파일명 추출
    filename = os.path.basename(png_file)
    # 새로운 저장 경로 생성
    output_path = os.path.join(output_directory, filename)
    # 파일 복사 (원하는 경로에 파일 저장)
    os.rename(png_file, output_path)

print("PNG 파일들이 성공적으로 저장되었습니다.")


PNG 파일들이 성공적으로 저장되었습니다.


In [ ]:
%cd /content/drive/MyDrive/yolov5

/content/drive/MyDrive/yolov5


In [ ]:
!python train.py --img 640 --batch 16 --epochs 30 --data /content/drive/MyDrive/ConSet/data.yaml --cfg models/yolov5s.yaml --weights "/content/drive/MyDrive/yolov5/runs/train/Conresult4/weights/best.pt"  --name Conresult5


WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

train: weights=/content/drive/MyDrive/yolov5/runs/train/Conresult4/weights/best.pt, cfg=models/yolov5s.yaml, data=/content/drive/MyDrive/ConSet/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/

In [ ]:
!python val.py --data "/content/drive/MyDrive/ConSet/data.yaml" --img 1280 --weights "/content/drive/MyDrive/yolov5/runs/train/Conresult5/weights/best.pt"

In [ ]:
!zip -r /content/drive/MyDrive/yolov5.zip /content/drive/MyDrive/yolov5

	zip warning: name not matched: /content/drive/MyDrive/yolov5

zip error: Nothing to do! (try: zip -r /content/drive/MyDrive/yolov5.zip . -i /content/drive/MyDrive/yolov5)
